In [1]:
# %cd ..

In [14]:
import torch
from PIL import Image, ImageDraw, ImageFont
import os

# Load your custom YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s_results/weights/best.pt')

# Function to calculate IoU (Intersection over Union)
def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)

    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area if union_area != 0 else 0

    return iou

# Function to perform detection, display results, and save the annotated image
def detect_and_save(image_path, save_path):

    font = ImageFont.truetype("/usr/share/fonts/truetype/fonts-yrsa-rasa/Rasa-Regular.ttf", 30)  # Adjusted font size
    # Load a larger font
    # try:
    #     font = ImageFont.truetype("arial.ttf", size=30)  # Adjusted font size
    # except IOError:
    #     font = ImageFont.load_default()
    #     print("Not join the condition")
    
    # Perform object detection on the image
    results = model(image_path)

    # Get the detection results as a pandas DataFrame
    df = results.pandas().xyxy[0]  # Bounding box format: xmin, ymin, xmax, ymax

    # Initialize the results dictionary
    output_results = {
        'sign': "NG",
        'duty_stamp': "NG",
        'e_duty_stamp': "NG",
        'IoU_sign_dstamp': "NG",
        'IoU_sign_e_dstamp': "NG",
        'Validation Status': "Not Accepted",
    }

    # Update the results based on detected labels
    detected_labels = df['name'].tolist()
    for label in output_results.keys():
        if label in detected_labels:
            output_results[label] = "OK"

    # Calculate IoU and update status based on IoU
    bboxes = {row['name']: [row['xmin'], row['ymin'], row['xmax'], row['ymax']] for _, row in df.iterrows()}
    iou_sign_duty = calculate_iou(bboxes.get('sign', [0, 0, 0, 0]), bboxes.get('duty_stamp', [0, 0, 0, 0]))
    iou_sign_e_duty = calculate_iou(bboxes.get('sign', [0, 0, 0, 0]), bboxes.get('e_duty_stamp', [0, 0, 0, 0]))

    iou_threshold = 0.10
    if iou_sign_duty > iou_threshold:
        output_results['IoU_sign_dstamp'] = "OK"

    if iou_sign_e_duty <= 0 and sum(bboxes.get('e_duty_stamp', [0, 0, 0, 0])) != 0:
        output_results['IoU_sign_e_dstamp'] = "OK"

    if output_results['IoU_sign_dstamp'] == "OK" and output_results['sign'] == "OK" and output_results['duty_stamp'] == "OK":
        output_results['Validation Status'] = "Accepted"
        
    if output_results['IoU_sign_e_dstamp'] == "OK" and output_results['sign'] == "OK" and output_results['e_duty_stamp'] == "OK":
        output_results['Validation Status'] = "Accepted"

    # Open the original image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    # Draw bounding boxes with labels
    for _, row in df.iterrows():
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        label = row['name']

        # Set the color based on the label
        if label == "sign":
            color = "blue"
        elif label == "duty_stamp":
            color = "orange"
        elif label == "e_duty_stamp":
            color = "red"
        else:
            color = "white"

        # Draw the bounding box
        draw.rectangle([xmin, ymin, xmax, ymax], outline=color, width=3)

        # Draw the label text
        text = label
        # Use textbbox to get the bounding box for the text
        text_bbox = draw.textbbox((xmin, ymin - 20), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

        # Draw background rectangle for text
        text_background = [xmin, ymin - text_height - 10, xmin + text_width, ymin]
        draw.rectangle(text_background, fill=color)

        # Draw text on top of rectangle
        draw.text((xmin + 2, ymin - text_height - 8), text, font=font, fill="white")

    # Position the text in the top-right corner
    text_x = 100
    text_y = 10
    padding = 20

    for label, status in output_results.items():
        text = f"{label}: {status}"

        if label == "IoU_sign_dstamp" or label == "IoU_sign_e_dstamp":
            text_color = 'white' if status == "OK" else 'black'
            bg_color = 'blue' if status == "OK" else 'lightcoral'

        elif label == "Validation Status":
            text_color = 'black' if status == "Accepted" else 'black'
            bg_color = 'green' if status == "Accepted" else 'lightcoral'
            
        else:
            text_color = 'black' if status == "OK" else 'black'
            bg_color = 'lightgreen' if status == "OK" else 'lightcoral'

        # Calculate text size and background size
        bbox = draw.textbbox((text_x, text_y), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        bg_width = text_width + 2 * padding
        bg_height = text_height + 2 * padding

        # Draw background rectangle
        draw.rectangle([text_x, text_y, text_x + bg_width, text_y + bg_height], fill=bg_color)

        # Draw text on top of rectangle
        draw.text((text_x + padding, text_y + padding), text, font=font, fill=text_color)
        text_y += bg_height  # Move to next line

    # Check if save directory exists, create if not
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Save the image with annotations
    image_filename = os.path.basename(image_path)
    saved_image_path = os.path.join(save_path, image_filename)
    image.save(saved_image_path)

    return saved_image_path

# Path to the image to test
image_path = 'extracted_data/Aby Ramadhani_page_3.png'

# Path to the directory to save the annotated image
save_path = './detect'

# Perform detection, display results, and save the annotated image
saved_image_path = detect_and_save(image_path, save_path)

print(f"Saved image with annotations: {saved_image_path}")


Using cache found in /home/mizanul/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-11-19 Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 3904MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7251912 parameters, 0 gradients
Adding AutoShape... 


Saved image with annotations: ./detect/Aby Ramadhani_page_3.png


# tampilkan image

In [ ]:

import os

# Path ke gambar yang ingin ditampilkan
saved_image_path = "detect/Aby Ramadhani_page_3.jpg"

# Fungsi untuk meresize gambar dengan mempertahankan rasio asli
def resize_image(image, scale_factor):
    # Ambil ukuran gambar asli
    width, height = image.size
    # Hitung ukuran baru
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    # Resize gambar
    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    return resized_image

# Pastikan file gambar ada
if os.path.exists(saved_image_path):
    # Baca gambar menggunakan PIL
    img = Image.open(saved_image_path)
    
    # Resize gambar dengan mengurangi ukuran menjadi 1/3
    img_resized = resize_image(img, scale_factor=1/3)
    
    # Tampilkan gambar menggunakan IPython display
    display(img_resized)
else:
    print(f"File tidak ditemukan: {saved_image_path}")

File tidak ditemukan: detect/Aby Ramadhani_page_3.jpg


In [ ]:
# %cd yolov5
!python3 detect.py --weights ../yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source "../annotated_dataset/test/images"

detect: weights=['../yolov5s_results/weights/best.pt'], source=../annotated_dataset/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-362-ge8a30cf8 Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 3904MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7251912 parameters, 0 gradients
image 1/21 /home/mizanul/Documents/code/document_validator/annotated_dataset/test/images/Ahmad-Arsyad-Surgi-Mukti_page_3_png.rf.5c2a8a2262c19b091db7d2db3eed6f72.jpg: 416x320 1 sign, 43.7ms
image 2/21 /home/mizanul/Documents/code/document_validator/annotated_dataset/test/images/Christine-